In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45265
2,Huelva,Confirmados PDIA 14 días,362
3,Huelva,Tasa PDIA 14 días,"70,54192567765068"
4,Huelva,Confirmados PDIA 7 días,167
5,Huelva,Tasa PDIA 7 dias,"32,5428220667615"
6,Huelva,Total Confirmados,45468
7,Huelva,Curados,44062
8,Huelva,Fallecidos,413


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45265.0


/tmp/ipykernel_34957/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12511.0


/tmp/ipykernel_34957/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_34957/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_34957/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_34957/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3785 personas en los últimos 7 días 

Un positivo PCR cada 1754 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45265.0,167.0,362.0,513170.0,32.542822,70.541926,24.0
Condado-Campiña,14055.0,67.0,126.0,156231.0,42.885215,80.649807,13.0
Huelva-Costa,26794.0,95.0,218.0,289548.0,32.809759,75.289762,11.0
Palos de la Frontera,1027.0,14.0,24.0,11742.0,119.230114,204.394481,5.0
Almonte,2280.0,14.0,25.0,24507.0,57.126535,102.011670,4.0
Cartaya,1962.0,12.0,35.0,20083.0,59.752029,174.276751,4.0
Huelva (capital),12511.0,38.0,82.0,143837.0,26.418793,57.008975,3.0
Bollullos Par del Condado,1277.0,15.0,18.0,14387.0,104.260791,125.112949,2.0
Isla Cristina,3114.0,6.0,20.0,21393.0,28.046557,93.488524,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Higuera de la Sierra,70.0,3.0,6.0,1291.0,232.378002,464.756003,0.0
Paterna del Campo,207.0,2.0,14.0,3457.0,57.853630,404.975412,0.0
Palos de la Frontera,1027.0,14.0,24.0,11742.0,119.230114,204.394481,5.0
Cartaya,1962.0,12.0,35.0,20083.0,59.752029,174.276751,4.0
Ayamonte,2067.0,17.0,36.0,21104.0,80.553450,170.583776,1.0
Bollullos Par del Condado,1277.0,15.0,18.0,14387.0,104.260791,125.112949,2.0
Almonte,2280.0,14.0,25.0,24507.0,57.126535,102.011670,4.0
Moguer,1951.0,8.0,22.0,21867.0,36.584808,100.608222,1.0
Isla Cristina,3114.0,6.0,20.0,21393.0,28.046557,93.488524,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,207.0,2.0,14.0,3457.0,57.853630,404.975412,0.0,0.142857
Rociana del Condado,665.0,1.0,4.0,7939.0,12.596045,50.384179,0.0,0.250000
Sierra de Huelva-Andévalo Central,3992.0,5.0,18.0,67391.0,7.419388,26.709798,0.0,0.277778
Isla Cristina,3114.0,6.0,20.0,21393.0,28.046557,93.488524,2.0,0.300000
Cartaya,1962.0,12.0,35.0,20083.0,59.752029,174.276751,4.0,0.342857
Lepe,3003.0,9.0,25.0,27880.0,32.281205,89.670014,0.0,0.360000
Moguer,1951.0,8.0,22.0,21867.0,36.584808,100.608222,1.0,0.363636
Huelva-Costa,26794.0,95.0,218.0,289548.0,32.809759,75.289762,11.0,0.435780
Huelva,45265.0,167.0,362.0,513170.0,32.542822,70.541926,24.0,0.461326
